In [2]:
import pandas as pd
import numpy as np
from sklearn.tree import DecisionTreeRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
indices_df = pd.read_csv('../Data/expanded_df_id_interpolated.csv')
print(indices_df.head())
stock_df = pd.read_csv('../Data/loreal_stock_price.csv')
print(stock_df.head())
currency_df = pd.read_csv('../Data/eur_usd_exchange_rate.csv')
print(currency_df.head())

FileNotFoundError: [Errno 2] No such file or directory: '../Data/expanded_df_id_interpolated.csv'

In [ ]:
# === Part 1: Data Preparation ===
print("=== Part 1: Data Preparation ===")

# Đọc dữ liệu (thay thế bằng file cụ thể của bạn)
# Dữ liệu giả định có cột 'Date', 'Close stock price', và các yếu tố ảnh hưởng khác
stock_df = pd.read_csv("stock_data.csv")

# Xử lý cột ngày tháng
stock_df['Date'] = pd.to_datetime(stock_df['Date'], errors='coerce', utc=True)
stock_df.dropna(subset=['Date'], inplace=True)
stock_df['Date'] = stock_df['Date'].dt.date

# Chỉ sử dụng các cột cần thiết
stock_df = stock_df[['Date', 'Close stock price']]
print("Sample data after preparation:")
print(stock_df.head())

# === Part 2: Feature Engineering ===
print("\n=== Part 2: Feature Engineering ===")

# Tạo lag features (giá cổ phiếu ngày trước đó)
look_back = 10  # Số ngày trước cần sử dụng để dự đoán
for lag in range(1, look_back + 1):
    stock_df[f'Lag_{lag}'] = stock_df['Close stock price'].shift(lag)

# Loại bỏ các hàng có giá trị NaN do tạo lag features
stock_df.dropna(inplace=True)

# Chia dữ liệu thành đặc trưng (X) và mục tiêu (y)
X = stock_df[[f'Lag_{lag}' for lag in range(1, look_back + 1)]]
y = stock_df['Close stock price']

print("Sample data with lag features:")
print(stock_df.head())

# === Part 3: Train-Test Split ===
print("\n=== Part 3: Train-Test Split ===")

# Chia dữ liệu thành tập huấn luyện và kiểm tra
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, shuffle=False)

print(f"Training set size: {X_train.shape[0]}, Test set size: {X_test.shape[0]}")

# === Part 4: Decision Tree Regressor ===
print("\n=== Part 4: Decision Tree Regressor ===")

# Khởi tạo và huấn luyện mô hình Decision Tree Regressor
decision_tree_model = DecisionTreeRegressor(random_state=42)
decision_tree_model.fit(X_train, y_train)

# Dự đoán trên tập kiểm tra
y_pred = decision_tree_model.predict(X_test)

# Đánh giá mô hình
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R^2 Score: {r2}")

# === Part 5: Predict Future Growth ===
print("\n=== Part 5: Predict Future Growth ===")

# Lấy dữ liệu cuối cùng để dự đoán 60 ngày tiếp theo
future_predictions = []
current_input = X.iloc[-1].values  # Sử dụng giá trị cuối cùng của tập dữ liệu

for _ in range(60):
    # Dự đoán giá trị tiếp theo
    next_prediction = decision_tree_model.predict([current_input])[0]
    future_predictions.append(next_prediction)

    # Cập nhật đầu vào cho dự đoán tiếp theo
    current_input = np.append(current_input[1:], next_prediction)

# Tạo DataFrame chứa dự đoán tương lai
future_dates = pd.date_range(start=stock_df['Date'].iloc[-1], periods=61, freq='D')[1:]
future_df = pd.DataFrame({'Date': future_dates, 'Predicted Price': future_predictions})

# === Part 6: Visualization ===
print("\n=== Part 6: Visualization ===")

plt.figure(figsize=(12, 6))
plt.plot(stock_df['Date'], stock_df['Close stock price'], label='Actual Prices')
plt.plot(future_df['Date'], future_df['Predicted Price'], label='Predicted Prices (Next 60 Days)', linestyle='--')
plt.legend()
plt.title("Stock Price Prediction for Next 60 Days (Decision Tree)")
plt.xlabel("Date")
plt.ylabel("Stock Price")
plt.show()


FileNotFoundError: [Errno 2] No such file or directory: '../Data/expanded_df_id_interpolated.csv'